In [22]:
import redis
import tweepy,pymongo
import json
import pandas as pd
from bson.json_util import dumps

In [23]:
client = pymongo.MongoClient()
db = client["tweet_database"]
tweets = db.tweets_collection
redis_db = redis.StrictRedis(host="localhost", port=6379, db=0)

## Creating cache with word covid19

In [24]:
myquery = {"content":{"$regex":"covid19","$options" :'i'}}
twts = list(tweets.find(myquery))
##Saving in cache
serializedObj = dumps(twts) #serialize object for the set redis.
result = redis_db.set('covid19_cache_dict', serializedObj) #set serialized object to redis server.

In [25]:
from bson.json_util import loads
json_string = redis_db.get('covid19_cache_dict')
covid19_dict = loads(json_string)
covid19_coll =  db["covid19_collection"]
covid19_coll.insert_many(covid19_dict)

In [28]:
import time
start_time = time.time()
twts = covid19_coll.find( {
    "$and" : [
        {"content":{"$regex":"covid19","$options" :'i'}} , {"content":{"$regex":"#trump","$options" :'i'}}
    ]
} )
twts = list(twts)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.003928184509277344 millie seconds ---


In [29]:
import time
start_time = time.time()
twts = tweets.find( {
    "$and" : [
        {"content":{"$regex":"covid19","$options" :'i'}} , {"content":{"$regex":"#trump","$options" :'i'}}
    ]
} )
twts = list(twts)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.03908705711364746 seconds ---


In [45]:
myquery = {"content":{"$regex":"#trump","$options" :'i'}}
twts = list(tweets.find(myquery))
print(len(twts))
print(twts[0])
##Saving in cache
serializedObj = dumps(twts) #serialize object for the set redis.
result = redis_db.set('trump_cache_dict', serializedObj) #set serialized object to redis server.

277
{'_id': ObjectId('5ea77cd2c9283bc56a26dbba'), 'id_str': '1254918713595191296', 'user_name': 'The Big Short', 'user_id': '1239693671965929472', 'content': '@ttmygh BREAKING NEWS! @realDonaldTrump drops Pence, adds @BernieSanders to the ticket to help Powell and Mnuchin fully socializing the Stock Mkt, Corporate America and the U.S. economy. #TrumpSanders 🤣', 'created_at': '2020-04-27 23:41:51', 'location': '', 'hashtags': ['TrumpSanders'], 'mentions': ['415921084', '25073877', '216776631'], 'in_reply_to_user_id': 415921084, 'in_reply_to_status_id': 1254874717854588930, 'retweetedFrom': -1, 'media': [], 'retweet': False, 'FavCount': 0, 'source': 'Twitter Web App', 'retweet_count': 0}


In [32]:
from bson.json_util import loads
json_string = redis_db.get('trump_cache_dict')
trump_dict = loads(json_string)
trump_coll =  db["trump_collection"]
trump_coll.insert_many(trump_dict)

In [46]:
## Tweet with  maximum retweet count
import time
start_time = time.time()
tweet_number = 0
for tweet in trump_coll.find():
    if tweet['FavCount']>4:
        tweet_number = tweet_number + 1
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.006793022155761719 seconds ---


In [47]:
import time
start_time = time.time()
tweet_number = 0
for tweet in tweets.find():
    if tweet['FavCount']>4 and ('trump' in tweet['hashtags']):
        tweet_number = tweet_number + 1
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.296842098236084 seconds ---


In [13]:
myquery = {}

mydoc = tweets.find(myquery)
count = 0
for x in mydoc:
    count = count + 1
print(count)

15569
